<a href="https://colab.research.google.com/github/SeungHan0816/WM-811k_Wafermap_Practice/blob/main/WM_811k_Wafermap_Practice02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
qingyi_wm811k_wafer_map_path = kagglehub.dataset_download('qingyi/wm811k-wafer-map')
print('Data source import complete.')

In [ ]:
import numpy as np # linear algebra(선형대수)
import pandas as pd # data processing, CSV file I/O(데이터 처리, CSV 파일 입출력)
import matplotlib.pyplot as plt
%matplotlib inline
import os # 운영체제 관련 기능을 다루는 라이브러리(파일 경로나 폴더 등을 다루고 싶을 때 사용)
print(qingyi_wm811k_wafer_map_path)
print(os.listdir(qingyi_wm811k_wafer_map_path))

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_pickle(os.path.join(qingyi_wm811k_wafer_map_path, "LSWMD.pkl"))
df.info() # df가 어떤 데이터를 담고 있는지 구조 요약을 출력

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
import matplotlib.pyplot as plt\
%matplotlib inline
uni_Index=np.unique(df.waferIndex, return_counts=True)
plt.bar(uni_Index[0],uni_Index[1], color='green', align='center', alpha=0.5)
plt.title("wafer index distribution")
plt.xlabel("index number")
plt.ylabel("freq")
plt.xlim(0,26)
plt.ylim(30000,34000)
plt.show() # 그래프 출력
lot_counts = df['lotName'].value_counts()
incomplete_lots = (lot_counts < 25).sum()
print("25개 미만인 lot 수:", incomplete_lots)
percent = (lot_counts < 25).mean() * 100
print(f"25개 미만인 lot 비율: {percent:.2f}%")

In [ ]:
df=df.drop(['waferIndex'], axis=1)

if 'waferIndex' in df.columns:
  df = df.drop(['waferIndex'], axis=1)
else:
  print("waferIndex 열이 이미 삭제되었습니다.")

print(df.columns)

In [ ]:
def find_dim(x):
    dim0=np.size(x, axis=0) # 행 개수
    dim1=np.size(x, axis=1) # 열 개수
    return dim0, dim1 # (행, 열) 튜플로 리턴
df['waferMapDim']=df.waferMap.apply(find_dim) # waferMap 열의 각 원소에 find_dim(x) 함수를 하나씩 적용
df.sample(5) # 무작위 5개 샘플을 출력

In [ ]:
max(df.waferMapDim), min(df.waferMapDim)

In [ ]:
uni_waferDim=np.unique(df.waferMapDim, return_counts=True)
uni_waferDim[0].shape[0]

In [ ]:
df['failureNum']=df.failureType
df['trainTestNum']=df.trianTestLabel
mapping_type={'Center':0, 'Donut':1, 'Edge-Loc':2, 'Edge-Ring':3, 'Loc':4, 'Random':5,'Scratch':6, 'Near-full':7, 'none':8}
mapping_traintest={'Training':0, 'Test':1}
df=df.replace({'failureNum':mapping_type, 'trainTestNum':mapping_traintest})

tol_wafers=df.shape[0] # 행 갯수 반환
tol_wafers

In [ ]:
df_withlabel = df[(df['failureNum']>=0) & (df['failureNum']<=8)] # 새로운 데이터 프레임 생성
df_withlabel = df_withlabel.reset_index() # 앞에서 생성한 데이터 프레임의 인덱스 재설정
df_withpattern = df[(df['failureNum']>=0) & (df['failureNum']<=7)] # 새로운 데이터 프레임 생성
df_withpattern = df_withpattern.reset_index() # 데이터 프레임 인덱스 재설정
df_nonpattern = df[(df['failureNum']==8)] # 새로운 데이터 프레임 생성
df_withlabel.shape[0], df_withpattern.shape[0], df_nonpattern.shape[0] # 각 데이터 프레임 행 갯수 반환

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import gridspec

figure = plt.figure(figsize=(15,3)) # 전체 그림 사이즈 설정
gridspec = gridspec.GridSpec(1, 2, width_ratios=[1,2]) # 1행 2열로, 비율 1:2로 나눔
ax1 = plt.subplot(gridspec[0]) # 파이 차트 (ax1) -> 레이블 없는 샘플 / 결함 샘플 / 정상 샘플의 비율
ax2 = plt.subplot(gridspec[1]) # 막대 차트 (ax2) -> 결함 유형별 분포 비율 (0~7번 클래스)
no_wafers=[tol_wafers-df_withlabel.shape[0], df_withpattern.shape[0], df_nonpattern.shape[0]]
colors = ['red', 'yellow', 'green'] # colors: 각각 빨강(라벨 없음), 노랑(결함 있음), 초록(정상)
explode = (0.1, 0, 0)
labels = ['no-label', 'label & pattern', 'label & non-pattern'] # 각 색별로 명칭 지정
ax1.pie(no_wafers, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
uni_pattern=np.unique(df_withpattern.failureNum, return_counts=True)
labels2=['','Center','Donut','Edge-Loc','Edge-Ring','Loc','Random','Scratch','Near-full']
ax2.bar(uni_pattern[0],uni_pattern[1]/df_withpattern.shape[0], color='yellow', align='center', alpha=1)
ax2.set_title("failure type freq")
ax2.set_ylabel("% of pattern wafers")
ax2.set_xticklabels(labels2) # x축 눈금 숫자 위치에 대응하는 labels2 이름들을 붙여줌.
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 10, ncols = 10, figsize=(30, 30))
ax = ax.ravel(order='C')
for i in range(100):
  img = df_withpattern.waferMap[i] # 결함 있는 웨이퍼 맵 이미지 한 개를 불러옴
  ax[i].imshow(img) # i번째 subplot에 방금 가져온 이미지를 시각화
  ax[i].set_title(df_withpattern.failureType[i][0][0], fontsize=15)
  ax[i].set_xlabel(df_withpattern.index[i], fontsize=10)
  ax[i].set_xticks([])
  ax[i].set_yticks([])
plt.tight_layout()
plt.show()

In [ ]:
x=[0,1,2,3,4,5,6,7] # 인덱스 8개
labels2=['Center','Donut','Edge-Loc','Edge-Ring','Loc','Random','Scratch','Near-full'] # 라벨 8개
for i in x: # 즉, 라벨 8종류 각각에 대해
  fig, ax=plt.subplots(nrows=1, ncols=10, figsize=(18,12))
  ax=ax.ravel(order='C') # ravel은 다차원 배열을 1차원으로 변경, C는 기본값
  for j in [i]:  # 그냥 j = i
    img=df_withpattern.waferMap[df_withpattern.failureType==labels2[j]]
    for k in range(10):
      ax[k].imshow(img[img.index[k]])
      ax[k].set_title(df_withpattern.failureType[img.index[k]][0][0], fontsize=10) # 타이틀
      ax[k].set_xlabel(df_withpattern.index[img.index[k]], fontsize=10) # x축에 인덱스 표시
      ax[k].set_xticks([])
      ax[k].set_yticks([])
    plt.tight_layout() # 자동 간격 조정
    plt.show() # 시각화

In [ ]:
x=[9, 340, 3, 16, 0, 25, 84, 37]
labels2=['Center','Donut','Edge-Loc','Edge-Ring','Loc','Random','Scratch','Near-full']
fig,ax=plt.subplots(nrows=2, ncols=4, figsize=(20,10))
ax=ax.ravel(order='C')
for i in range(8):
    img=df_withpattern.waferMap[x[i]]
    ax[i].imshow(img)
    ax[i].set_title(df_withpattern.failureType[x[i]][0][0], fontsize=24)
    ax[i].set_xticks([])
    ax[i].set_yticks([])
plt.tight_layout()
plt.show()

In [ ]:
import skimage
from skimage import measure
from skimage.transform import radon
from skimage.transform import probabilistic_hough_line
from scipy import interpolate
from scipy import stats

an=np.linspace(0, 2*np.pi, 100)
plt.plot(2.5*np.cos(an), 2.5*np.sin(an))
plt.axis('equal')
plt.axis([-4,4,-4,4])

plt.plot([-2.5, 2.5], [1.5, 1.5])
plt.plot([-2.5, 2.5], [0.5, 0.5 ])
plt.plot([-2.5, 2.5], [-0.5, -0.5 ])
plt.plot([-2.5, 2.5], [-1.5,-1.5 ])

plt.plot([0.5, 0.5], [-2.5, 2.5])
plt.plot([1.5, 1.5], [-2.5, 2.5])
plt.plot([-0.5, -0.5], [-2.5, 2.5])
plt.plot([-1.5, -1.5], [-2.5, 2.5])

plt.title("Divide wafer map to 13 regions")
plt.xticks([]) # 축 없애기
plt.yticks([])
plt.show()

In [ ]:
def cal_den(x):
  return 100*(np.sum(x==2)/np.size(x))

def find_regions(x):
  rows=np.size(x,axis=0) # rows: 행 수 (높이) -> np.size(x, axis=0) = x.shape[0]
  cols=np.size(x,axis=1) # cols: 열 수 (너비) -> np.size(x, axis=1) = x.shape[1]
  ind1=np.arange(0,rows,rows//5)
  ind2=np.arange(0,cols,cols//5)

  # 가장자리 영역
  reg1=x[ind1[0]:ind1[1],:] # 위쪽 행만 모든 열 가져옴
  reg3=x[ind1[4]:,:] # 아래쪽 행만 모든 열 가져옴
  reg4=x[:,ind2[0]:ind2[1]] # 모든 행, 왼쪽 열만 가져옴
  reg2=x[:,ind2[4]:] # 모든 행, 오른쪽 열만 가져옴

  # 중앙 3x3 격자
  reg5=x[ind1[1]:ind1[2],ind2[1]:ind2[2]]
  reg6=x[ind1[1]:ind1[2],ind2[2]:ind2[3]]
  reg7=x[ind1[1]:ind1[2],ind2[3]:ind2[4]]
  reg8=x[ind1[2]:ind1[3],ind2[1]:ind2[2]]
  reg9=x[ind1[2]:ind1[3],ind2[2]:ind2[3]]
  reg10=x[ind1[2]:ind1[3],ind2[3]:ind2[4]]
  reg11=x[ind1[3]:ind1[4],ind2[1]:ind2[2]]
  reg12=x[ind1[3]:ind1[4],ind2[2]:ind2[3]]
  reg13=x[ind1[3]:ind1[4],ind2[3]:ind2[4]]

  fea_reg_den = []
  fea_reg_den = [cal_den(reg1),cal_den(reg2),cal_den(reg3),cal_den(reg4),cal_den(reg5),cal_den(reg6),cal_den(reg7),cal_den(reg8),cal_den(reg9),cal_den(reg10),cal_den(reg11),cal_den(reg12),cal_den(reg13)]
  return fea_reg_den

df_withpattern['fea_reg']=df_withpattern.waferMap.apply(find_regions)

x = [9,340, 3, 16, 0, 25, 84, 37]
labels2 = ['Center','Donut','Edge-Loc','Edge-Ring','Loc','Random','Scratch','Near-full']

fig, ax = plt.subplots(nrows = 2, ncols = 4,figsize=(20, 10))
ax = ax.ravel(order='C')
for i in range(8):

  ax[i].bar(np.linspace(1,13,13),df_withpattern.fea_reg[x[i]])
  ax[i].set_title(df_withpattern.failureType[x[i]][0][0],fontsize=15)
  ax[i].set_xticks([])
  ax[i].set_yticks([])
plt.tight_layout()
plt.show()

In [ ]:
def change_val(img):
    img[img==1] =0 # 조건에 해당하는 위치(==1)의 값을 0으로 바꾼다
    return img # 값이 바뀐 이미지 결과를 리턴
df_withpattern_copy = df_withpattern.copy()
df_withpattern_copy['new_waferMap'] =df_withpattern_copy.waferMap.apply(change_val)

x = [9,340, 3, 16, 0, 25, 84, 37]
labels2 = ['Center','Donut','Edge-Loc','Edge-Ring','Loc','Random','Scratch','Near-full']

fig, ax = plt.subplots(nrows = 2, ncols = 4, figsize=(20, 10))
ax = ax.ravel(order='C')
for i in range(8):
  img = df_withpattern_copy.waferMap[x[i]]
  theta = np.linspace(0., 180., max(img.shape), endpoint=False)
  sinogram = radon(img, theta=theta)
  ax[i].imshow(sinogram, cmap=plt.cm.Greys_r, extent=(0, 180, 0, sinogram.shape[0]), aspect='auto')
  ax[i].set_title(df_withpattern_copy.failureType[x[i]][0][0],fontsize=15)
  ax[i].set_xticks([]) # x축 눈금 제거
plt.tight_layout() # 그래프 간 간격 자동 조정
plt.show()

In [ ]:
def cubic_inter_mean(img):
  theta = np.linspace(0., 180., max(img.shape), endpoint=False)
  sinogram = radon(img, theta=theta)
  xMean_Row = np.mean(sinogram, axis = 1)
  x = np.linspace(1, xMean_Row.size, xMean_Row.size)
  y = xMean_Row
  f = interpolate.interp1d(x, y, kind = 'cubic')
  xnew = np.linspace(1, xMean_Row.size, 20)
  ynew = f(xnew)/100
  return ynew # 20개의 평균값 배열 반환

def cubic_inter_std(img):
  theta = np.linspace(0., 180., max(img.shape), endpoint=False)
  sinogram = radon(img, theta=theta)
  xStd_Row = np.std(sinogram, axis=1)
  x = np.linspace(1, xStd_Row.size, xStd_Row.size)
  y = xStd_Row
  f = interpolate.interp1d(x, y, kind = 'cubic')
  xnew = np.linspace(1, xStd_Row.size, 20) # 20개 보간
  ynew = f(xnew)/100   # use interpolation function returned by `interp1d`
  return ynew # 20개짜리 벡터 반환

df_withpattern_copy['fea_cub_mean'] =df_withpattern_copy.waferMap.apply(cubic_inter_mean)
df_withpattern_copy['fea_cub_std'] =df_withpattern_copy.waferMap.apply(cubic_inter_std)

x = [9,340, 3, 16, 0, 25, 84, 37]
labels2 = ['Center','Donut','Edge-Loc','Edge-Ring','Loc','Random','Scratch','Near-full']

fig, ax = plt.subplots(nrows = 2, ncols = 4,figsize=(20, 10))
ax = ax.ravel(order='C')
for i in range(8):
    ax[i].bar(np.linspace(1,20,20),df_withpattern_copy.fea_cub_mean[x[i]])
    ax[i].set_title(df_withpattern_copy.failureType[x[i]][0][0],fontsize=10)
    ax[i].set_xticks([])
    ax[i].set_xlim([0,21]) # x축 범위를 0~21로 고정(여유를 두기 위함)
    ax[i].set_ylim([0,1]) # y축 범위를 0~1로 고정
plt.tight_layout()
plt.show()

In [ ]:
x = [9,340, 3, 16, 0, 25, 84, 37]
labels2 = ['Center','Donut','Edge-Loc','Edge-Ring','Loc','Random','Scratch','Near-full']

fig, ax = plt.subplots(nrows = 2, ncols = 4,figsize=(20, 10))
ax = ax.ravel(order='C')
for i in range(8):
  img = df_withpattern_copy.waferMap[x[i]]
  zero_img = np.zeros(img.shape)
  img_labels = measure.label(img, connectivity=1, background=0)
  img_labels = img_labels-1

  if img_labels.max()==0:
    no_region = 0
  else:
    info_region = stats.mode(img_labels[img_labels>-1], axis = None)
    no_region = info_region[0]
  zero_img[np.where(img_labels==no_region)] = 2
  ax[i].imshow(zero_img) # zero_img를 해당 subplot 위치에 표시
  ax[i].set_title(df_withpattern_copy.failureType[x[i]][0][0],fontsize=10)
  ax[i].set_xticks([]) # x축 눈금 제거
plt.tight_layout() # 그래프끼리 겹치지 않게 자동으로 레이아웃 조정
plt.show()

In [ ]:
def cal_dist(img,x,y):
  dim0=np.size(img,axis=0) # dim0: 이미지의 세로 길이 (행의 수)
  dim1=np.size(img,axis=1) # dim1: 이미지의 가로 길이 (열의 수)
  dist = np.sqrt((x-dim0/2)**2+(y-dim1/2)**2)
  return dist

def fea_geom(img):
  norm_area=img.shape[0]*img.shape[1]
  norm_perimeter=np.sqrt((img.shape[0])**2+(img.shape[1])**2)
  img_labels = measure.label(img, connectivity=1, background=0)
  if img_labels.max()==0:
    img_labels[img_labels==0]=1
    no_region = 0
  else:
    info_region = stats.mode(img_labels[img_labels>0], axis = None)
    no_region = info_region[0]-1

  prop = measure.regionprops(img_labels)
  prop_area = prop[no_region].area/norm_area
  prop_perimeter = prop[no_region].perimeter/norm_perimeter
  prop_cent = prop[no_region].local_centroid
  prop_cent = cal_dist(img,prop_cent[0],prop_cent[1])
  prop_majaxis = prop[no_region].major_axis_length/norm_perimeter
  prop_minaxis = prop[no_region].minor_axis_length/norm_perimeter
  prop_ecc = prop[no_region].eccentricity
  prop_solidity = prop[no_region].solidity
  return prop_area,prop_perimeter,prop_majaxis,prop_minaxis,prop_ecc,prop_solidity

df_withpattern_copy['fea_geom'] =df_withpattern_copy.waferMap.apply(fea_geom)

df_withpattern_copy.fea_geom[340] #donut

In [ ]:
df_all=df_withpattern_copy.copy()
a=[df_all.fea_reg[i] for i in range(df_all.shape[0])] #13 -> "각 벡터는 13개의 값으로 구성되어 있다"
b=[df_all.fea_cub_mean[i] for i in range(df_all.shape[0])] #20 ->  "각 행마다 20개의 평균값 특징이 존재한다"
c=[df_all.fea_cub_std[i] for i in range(df_all.shape[0])] #20 -> "각 행마다 20개의 표준편차 특징이 존재한다"
d=[df_all.fea_geom[i] for i in range(df_all.shape[0])] #6 -> "각 행마다 6개의 기하학적 특징값이 있다"
fea_all = np.concatenate((np.array(a),np.array(b),np.array(c),np.array(d)),axis=1) #59 in total -> "총 59개의 특징값(열)을 가진다"

label=[df_all.failureNum[i] for i in range(df_all.shape[0])]
label=np.array(label)

from sklearn.model_selection import train_test_split
X = fea_all # 입력 데이터 (59개의 특성 벡터) -> 넘파이 배열 (샘플 수, 59)
y = label # 정답 데이터 (결함 번호) -> 정수값으로 구성된 넘파이 배열 (샘플 수,)
from collections import  Counter
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print('Training target statistics: {}'.format(Counter(y_train)))
print('Testing target statistics: {}'.format(Counter(y_test)))

RANDOM_STATE =42 # 랜덤 시드를 상수로 따로 저장 -> 여러 곳에서 동일한 시드 사용할 때 편함

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier
import numpy as np # 누락되어 있었던 경우 대비

clf2 = OneVsOneClassifier(LinearSVC(random_state = RANDOM_STATE)).fit(X_train, y_train)
y_train_pred = clf2.predict(X_train)
y_test_pred = clf2.predict(X_test)
train_acc2 = (y_train == y_train_pred).astype(float).sum() / X_train.shape[0]
test_acc2 = (y_test == y_test_pred).astype(float).sum() / X_test.shape[0]

print('One-Vs-One Training acc: {}'.format(train_acc2*100)) #One-Vs-One Training acc: 80.36 -> 82.73
print('One-Vs-One Testing acc: {}'.format(test_acc2*100)) #One-Vs-One Testing acc: 79.04 -> 82.36
print("y_train_pred[:100]: ", y_train_pred[:100])
print ("y_train[:100]: ", y_train[:100])

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

def plot_confusion_matrix(cm, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):

  # 이 함수는 혼동 행렬(confusion matrix)을 출력하고 시각화합니다.
  # 정규화를 적용하려면 `normalize=True`로 설정하면 됩니다.
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting `normalize=True`.
  """

  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Normalized confusion matrix") # 정규화된 혼동 행렬
  else:
    print('Confusion matrix, without normalization') # 정규화 없는 혼동 행렬

  print(cm) # 주석 처리된 코드. 혼동 행렬 자체를 텍스트로 출력하고 싶을 때 쓰면 됨.

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title) # 그래프 제목 지정
  plt.colorbar() # 색상 값 범례(옆에 색 막대) 추가

  fmt = '.2f' if normalize else 'd' # 값 표시 형식 지정
  thresh = cm.max() / 2. # 색 대비 기준 임계값

  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout() # 그래프 요소 간 여백 자동 조정
  plt.ylabel('True label') #  Y축 라벨: 실제 값
  plt.xlabel('Predicted label') # X축 라벨: 예측 값

cnf_matrix = confusion_matrix(y_test, y_test_pred)
np.set_printoptions(precision=2)
from matplotlib import gridspec
fig = plt.figure(figsize=(15, 8))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1])
plt.subplot(gs[0])
plot_confusion_matrix(cnf_matrix, title='Confusion matrix')
plt.subplot(gs[1])
plot_confusion_matrix(cnf_matrix, normalize=True, title='Normalized confusion matrix')
plt.show()